In [2]:
import os
import numpy as np
import cv2 as cv

# Process raw data


In [3]:
# Path of the directory with raw images
raw_dir = '/kaggle/input/mrl-dataset/train'
folders = ['Open_Eyes', 'Closed_Eyes']

# Path of the directory where processed images will be stored
processed_dir = '/kaggle/working'


In [4]:
# Mean and std of ImageNet will be used to normalize the images
mean = np.array([0.485, 0.456, 0.406])
std  = np.array([0.229, 0.224, 0.225])

images = []
labels = []

def process_image(img_path):
    img = cv.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}.")
        return None
    # Resize image to 224 x 224 as this image size is expected by ResNet or MobileNet which will be used for image classification
    img = cv.resize(img, (224, 224))
    # cv2 uses the BGR color format, so we convert it to the RGB format
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    # Normalize the image values to range [0, 1]
    img = img / 255.0
    img = (img - mean) / std
    return img


for label, folder in enumerate(folders):
    folder_path = os.path.join(raw_dir, folder)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        img = process_image(file_path)
        if img is not None:
            images.append(img)
            labels.append(label)

# Convert images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Save processed images and labels
np.save(os.path.join(processed_dir, "images.npy"), images)
np.save(os.path.join(processed_dir, "labels.npy"), labels)

In [5]:
from sklearn.model_selection import train_test_split

# Load the processed dataset
images = np.load(os.path.join(processed_dir, "images.npy"))
labels = np.load(os.path.join(processed_dir, "labels.npy"))

'''Split the dataset into train+val (80%) and test datasets (20%). The data is automatically shuffled. 
stratify=labels: the generated splits gave the same proprotion of labels as given by parameter "labels".'''
images_train_val, images_test, labels_train_val, labels_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

# Split the train_val dataset into train (75%) and val datasets (25%)
images_train, images_val, labels_train, labels_val = train_test_split(images_train_val, labels_train_val, test_size=0.25, random_state=42, stratify=labels_train_val)

print(f"Train set size: {images_train.shape[0]}")
print(f"Validation set size: {images_val.shape[0]}")
print(f"Test set size: {images_test.shape[0]}")

# Save split data and labels
np.save(os.path.join(processed_dir, "train_images.npy"), images_train)
np.save(os.path.join(processed_dir, "train_labels.npy"), labels_train)
np.save(os.path.join(processed_dir, "val_images.npy"), images_val)
np.save(os.path.join(processed_dir, "val_labels.npy"), labels_val)
np.save(os.path.join(processed_dir, "test_images.npy"), images_test)
np.save(os.path.join(processed_dir, "test_labels.npy"), labels_test)

Train set size: 2400
Validation set size: 800
Test set size: 800


# Finetune ResNet50

In [8]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models, layers, Sequential

def get_resnet50_model(variant='base'):
    # Load the ResNet50 model pretrained on ImageNet
    base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))
    train_from = 150

    if variant == 'finetune':
        base_model.trainable = True
        for layer in base_model.layers[:train_from]:
            layer.trainable = False
    else:
        # Freeze all layers
        base_model.trainable = False
    
    model_append = [layers.GlobalAveragePooling2D()]
    model_append.append(layers.Dense(1, activation='sigmoid'))
    model = Sequential([base_model] + model_append)

    return model

# Train

In [9]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC, F1Score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# Load train dataset
X_train = np.load(os.path.join(processed_dir, "train_images.npy"))
y_train = np.load(os.path.join(processed_dir, "train_labels.npy"))
# Reshape y_train (num_train_examples,) to have the same shape as model predictions in tf (num_train_examples, 1)
y_train = y_train.reshape(-1, 1)

# Load validation dataset
X_val = np.load(os.path.join(processed_dir, "val_images.npy"))
y_val = np.load(os.path.join(processed_dir, "val_labels.npy"))
# Reshape y_val (num_val_examples,) to have the same shape as model predictions in tf (num_val_examples, 1)
y_val = y_val.reshape(-1, 1)

# Choose model variant: 'base', 'finetune'
model_variant = 'finetune'
# Load ResNet50 model
model = get_resnet50_model(variant=model_variant)

# Configure model settings for training
model.compile(optimizer=Adam(learning_rate=1e-4), loss=BinaryCrossentropy(), metrics=[BinaryAccuracy(), Precision(), Recall(), AUC(), F1Score(threshold=0.5)])

# Callbacks
# Create the EarlyStopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint(f"/kaggle/working/{model_variant}.h5", save_best_only=True)

# Train modified ResNet50
history = model.fit(X_train, y_train, validation_data=[X_val, y_val], batch_size=32, epochs=15, callbacks=[early_stop, checkpoint])

I0000 00:00:1752495591.672439      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752495591.673147      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15


I0000 00:00:1752495621.758063     103 service.cc:148] XLA service 0x79cfb8003b50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752495621.759286     103 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752495621.759315     103 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752495623.866618     103 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/75 ━━━━━━━━━━━━━━━━━━━━ 34:31 28s/step - auc: 0.7314 - binary_accuracy: 0.7188 - f1_score: 0.7692 - loss: 0.6539 - precision: 0.6250 - recall: 1.0000

I0000 00:00:1752495632.033940     103 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


75/75 ━━━━━━━━━━━━━━━━━━━━ 47s 259ms/step - auc: 0.9742 - binary_accuracy: 0.9260 - f1_score: 0.9297 - loss: 0.1814 - precision: 0.9107 - recall: 0.9517 - val_auc: 0.9991 - val_binary_accuracy: 0.6175 - val_f1_score: 0.3806 - val_loss: 0.4657 - val_precision: 1.0000 - val_recall: 0.2350
Epoch 2/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 149ms/step - auc: 0.9999 - binary_accuracy: 0.9959 - f1_score: 0.9959 - loss: 0.0179 - precision: 0.9948 - recall: 0.9970 - val_auc: 0.9936 - val_binary_accuracy: 0.8737 - val_f1_score: 0.8879 - val_loss: 0.3512 - val_precision: 0.7984 - val_recall: 1.0000
Epoch 3/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 152ms/step - auc: 0.9997 - binary_accuracy: 0.9898 - f1_score: 0.9896 - loss: 0.0236 - precision: 0.9948 - recall: 0.9845 - val_auc: 1.0000 - val_binary_accuracy: 0.9987 - val_f1_score: 0.9988 - val_loss: 0.0059 - val_precision: 0.9975 - val_recall: 1.0000
Epoch 4/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 142ms/step - auc: 0.9989 - binary_accuracy: 0.9942 - f1_score: 0.9940 - 

# Test

In [10]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix

# Load test dataset
X_test = np.load("/kaggle/working/test_images.npy")
y_test = np.load("/kaggle/working/test_labels.npy")
# Reshape y_test (num_test_examples,) to have the same shape as model predictions in tf (num_test_examples, 1)
y_test = y_test.reshape(-1, 1)

# Load the updated resnet model
model = load_model("/kaggle/working/finetune.h5")

# Evaluate on the test dataset
results = model.evaluate(X_test, y_test, batch_size=32, verbose=1)

# Predict labels
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Open", "Closed"]))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 93ms/step - auc: 1.0000 - binary_accuracy: 0.9993 - f1_score: 0.9993 - loss: 8.1286e-04 - precision: 1.0000 - recall: 0.9985
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 90ms/step

Classification Report:
              precision    recall  f1-score   support

        Open       1.00      1.00      1.00       400
      Closed       1.00      1.00      1.00       400

    accuracy                           1.00       800
   macro avg       1.00      1.00      1.00       800
weighted avg       1.00      1.00      1.00       800


Confusion Matrix:
[[400   0]
 [  1 399]]
